

1.   Đọc dữ liệu




In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Đọc dữ liệu xlsx
file_path = "Online Retail.xlsx"
df = pd.read_excel(file_path)

print("Huỳnh Thiên Huy - 6351071028")
print(df.shape)
print(df.head())

Huỳnh Thiên Huy - 6351071028
(541909, 8)
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


2. Tiền xử lý

In [5]:
# Xóa các dòng có giá trị thiếu trong cột quan trọng (CustomerID)
df = df.dropna(subset=['CustomerID'])

# Loại bỏ các hóa đơn bị hủy (InvoiceNo bắt đầu bằng 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Loại bỏ dữ liệu có Quantity <= 0 hoặc UnitPrice <= 0
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Chuyển kiểu dữ liệu về đúng định dạng
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['CustomerID'] = df['CustomerID'].astype(int)

# Tạo cột doanh thu từng dòng (Quantity * UnitPrice)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Chuẩn hóa tên quốc gia (loại bỏ khoảng trắng dư thừa)
df['Country'] = df['Country'].str.strip()

# Reset lại index sau khi lọc
df.reset_index(drop=True, inplace=True)

print("\nSau khi xử lý:", df.shape)
print(df.head())
print("Huỳnh Thiên Huy - 6351071028")



Sau khi xử lý: (397884, 9)
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  TotalPrice  
0 2010-12-01 08:26:00       2.55       17850  United Kingdom       15.30  
1 2010-12-01 08:26:00       3.39       17850  United Kingdom       20.34  
2 2010-12-01 08:26:00       2.75       17850  United Kingdom       22.00  
3 2010-12-01 08:26:00       3.39       17850  United Kingdom       20.34  
4 2010-12-01 08:26:00       3.39       17850  United Kingdom       20.34  
Huỳnh Thiên Huy - 6351071028


3. Xóa các hóa đơn tín dụng

In [6]:
# Xóa các hóa đơn tín dụng (bắt đầu bằng 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Reset lại index sau khi lọc
df.reset_index(drop=True, inplace=True)

# Kiểm tra kết quả
print("Sau khi xóa hóa đơn tín dụng:", df.shape)
print(df['InvoiceNo'].head())


Sau khi xóa hóa đơn tín dụng: (397884, 9)
0    536365
1    536365
2    536365
3    536365
4    536365
Name: InvoiceNo, dtype: object


4. Thống kê số dòng dữ liệu theo từng quốc gia

In [7]:
country_counts = df['Country'].value_counts()
print(country_counts)


Country
United Kingdom          354321
Germany                   9040
France                    8341
EIRE                      7236
Spain                     2484
Netherlands               2359
Belgium                   2031
Switzerland               1841
Portugal                  1462
Australia                 1182
Norway                    1071
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         57
Lebanon                     45


5. Lấy dữ liệu từ United Kingdom

In [8]:
# Lọc dữ liệu chỉ lấy các giao dịch ở Anh
df_uk = df[df['Country'] == 'United Kingdom']

# Gom nhóm theo số hóa đơn và tên sản phẩm, tính tổng số lượng mua
grouped_uk = (
    df_uk.groupby(['InvoiceNo', 'Description'])['Quantity']
    .sum()
    .reset_index()
)

# Hiển thị vài dòng đầu tiên để kiểm tra
print(grouped_uk.head(10))
print("Số lượng nhóm:", grouped_uk.shape)


   InvoiceNo                          Description  Quantity
0     536365       CREAM CUPID HEARTS COAT HANGER         8
1     536365    GLASS STAR FROSTED T-LIGHT HOLDER         6
2     536365  KNITTED UNION FLAG HOT WATER BOTTLE         6
3     536365       RED WOOLLY HOTTIE WHITE HEART.         6
4     536365         SET 7 BABUSHKA NESTING BOXES         2
5     536365   WHITE HANGING HEART T-LIGHT HOLDER         6
6     536365                  WHITE METAL LANTERN         6
7     536366            HAND WARMER RED POLKA DOT         6
8     536366               HAND WARMER UNION JACK         6
9     536367        ASSORTED COLOUR BIRD ORNAMENT        32
Số lượng nhóm: (344342, 3)


6. Chuyển đổi dữ liệu về dạng hot encoding

In [9]:
# Gom nhóm theo hóa đơn và sản phẩm, tính tổng số lượng
basket = (
    df_uk.groupby(['InvoiceNo', 'Description'])['Quantity']
    .sum()
    .unstack()
    .fillna(0)
)

# Chuyển về dạng nhị phân (1 nếu có mua, 0 nếu không)
basket = basket.map(lambda x: 1 if x > 0 else 0)

# Kiểm tra kết quả
print(basket.head())
print("Kích thước dữ liệu sau One-Hot Encoding:", basket.shape)


Description   4 PURPLE FLOCK DINNER CANDLES   50'S CHRISTMAS GIFT BAG LARGE  \
InvoiceNo                                                                     
536365                                    0                               0   
536366                                    0                               0   
536367                                    0                               0   
536368                                    0                               0   
536369                                    0                               0   

Description   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK  \
InvoiceNo                                                       
536365                        0                             0   
536366                        0                             0   
536367                        0                             0   
536368                        0                             0   
536369                        0                         